### 英雄联盟赛事预测

英雄联盟（League of Legends，LoL）是一个多人在线竞技游戏，由拳头游戏（Riot Games）公司出品。在游戏中，每位玩家控制一位有独特技能的英雄，红蓝两支队伍各有五位玩家进行对战，目标是摧毁对方的基地水晶。水晶有多座防御塔保护，通常需要先摧毁一些防御塔再摧毁水晶。玩家所控制的英雄起初非常弱，需要不断击杀小兵、野怪和对方英雄来获得金币、经验。经验可以提升英雄等级和技能等级，金币可以用来购买装备提升攻击、防御等属性。对战过程中一般没有己方单位在附近的地点是没有视野的，即无法看到对面单位，双方可以通过使用守卫来监视某个地点，洞察对面走向、制定战术。
本数据集来自[Kaggle](https://www.kaggle.com/bobbyscience/league-of-legends-diamond-ranked-games-10-min)，包含了9879场钻一到大师段位的单双排对局，对局双方几乎是同一水平。每条数据是前10分钟的对局情况，每支队伍有19个特征，红蓝双方共38个特征。这些特征包括英雄击杀、死亡，金钱、经验、等级情况等等。一局游戏一般会持续30至40分钟，但是实际前10分钟的局面很大程度上影响了之后胜负的走向。作为最成功的电子竞技游戏之一，对局数据、选手数据的量化与研究具有重要意义，可以启发游戏将来的发展和改进。

1. 加载工具包

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
RANDOM_SEED = 2020

2. 导入数据集

In [29]:
data_path = "high_diamond_ranked_10min.csv"
data_df = pd.read_csv(data_path,sep=',')
data_df = data_df.drop(columns='gameId')

In [27]:
#数据预览
print(data_df.iloc[0])    #预览row 1 
data_df.describe()      #输出数据统计信息

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueFirstBlood                      1.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalExperience             17039.0
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueGoldDiff                      643.0
blueExperienceDiff                 -8.0
blueCSPerMin                       19.5
blueGoldPerMin                   1721.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redFirstBlood                       0.0
redKills                            6.0
redDeaths                           9.0


blueWins  blueWardsPlaced  blueWardsDestroyed  blueFirstBlood   
count  9879.000000      9879.000000         9879.000000     9879.000000  \
mean      0.499038        22.288288            2.824881        0.504808   
std       0.500024        18.019177            2.174998        0.500002   
min       0.000000         5.000000            0.000000        0.000000   
25%       0.000000        14.000000            1.000000        0.000000   
50%       0.000000        16.000000            3.000000        1.000000   
75%       1.000000        20.000000            4.000000        1.000000   
max       1.000000       250.000000           27.000000        1.000000   

         blueKills   blueDeaths  blueAssists  blueEliteMonsters  blueDragons   
count  9879.000000  9879.000000  9879.000000        9879.000000  9879.000000  \
mean      6.183925     6.137666     6.645106           0.549954     0.361980   
std       3.011028     2.933818     4.064520           0.625527     0.480597   
min       0.000000     0.000000     0.000000           0.000000     0.000000   
25%       4.000000     4.000000     4.000000           0.000000     0.000000   
50%       6.000000     6.000000     6.000000           0.000000     0.000000   
75%       8.000000     8.000000     9.000000           1.000000     1.000000   
max      22.000000    22.000000    29.000000           2.000000     1.000000   

       blueHeralds  ...  redTowersDestroyed  redTotalGold  redAvgLevel   
count  9879.000000  ...         9879.000000   9879.000000  9879.000000  \
mean      0.187974  ...            0.043021  16489.041401     6.925316   
std       0.390712  ...            0.216900   1490.888406     0.305311   
min       0.000000  ...            0.000000  11212.000000     4.800000   
25%       0.000000  ...            0.000000  15427.500000     6.800000   
50%       0.000000  ...            0.000000  16378.000000     7.000000   
75%       0.000000  ...            0.000000  17418.500000     7.200000   
max       1.000000  ...            2.000000  22732.000000     8.200000   

       redTotalExperience  redTotalMinionsKilled  redTotalJungleMinionsKilled   
count         9879.000000            9879.000000                  9879.000000  \
mean         17961.730438             217.349226                    51.313088   
std           1198.583912              21.911668                    10.027885   
min          10465.000000             107.000000                     4.000000   
25%          17209.500000             203.000000                    44.000000   
50%          17974.000000             218.000000                    51.000000   
75%          18764.500000             233.000000                    57.000000   
max          22269.000000             289.000000                    92.000000   

        redGoldDiff  redExperienceDiff  redCSPerMin  redGoldPerMin  
count   9879.000000        9879.000000  9879.000000    9879.000000  
mean     -14.414111          33.620306    21.734923    1648.904140  
std     2453.349179        1920.370438     2.191167     149.088841  
min   -11467.000000       -8348.000000    10.700000    1121.200000  
25%    -1596.000000       -1212.000000    20.300000    1542.750000  
50%      -14.000000          28.000000    21.800000    1637.800000  
75%     1585.500000        1290.500000    23.300000    1741.850000  
max    10830.000000        9333.000000    28.900000    2273.200000  

[8 rows x 39 columns]

3. 数据预处理

In [30]:
## 增删特征
drop_features = ['blueCSPerMin','redCSPerMin',
                 'blueGoldDiff','redGoldDiff',
                 'blueGoldPerMin','redGoldPerMin',
                 'blueTotalExperience','redTotalExperience',
                 'blueFirstBlood','redFirstBlood']
data_df = data_df.drop(columns=drop_features)       #丢掉重复且多余的特征

## 构造新的差值特征
info_names = [c[3:] for c in data_df.columns if c[:3] == 'red']
for name  in info_names:
    data_df['br' + name] = data_df['blue' + name] - data_df['red' + name]   #将新的差值特征并入datafram
data_df.iloc[0]

blueWins                            0.0
blueWardsPlaced                    28.0
blueWardsDestroyed                  2.0
blueKills                           9.0
blueDeaths                          6.0
blueAssists                        11.0
blueEliteMonsters                   0.0
blueDragons                         0.0
blueHeralds                         0.0
blueTowersDestroyed                 0.0
blueTotalGold                   17210.0
blueAvgLevel                        6.6
blueTotalMinionsKilled            195.0
blueTotalJungleMinionsKilled       36.0
blueExperienceDiff                 -8.0
redWardsPlaced                     15.0
redWardsDestroyed                   6.0
redKills                            6.0
redDeaths                           9.0
redAssists                          8.0
redEliteMonsters                    0.0
redDragons                          0.0
redHeralds                          0.0
redTowersDestroyed                  0.0
redTotalGold                    16567.0


In [34]:
## 特征离散化
discreate_n = 10    #将连续数值分为10个区间
discreate_df = data_df.copy()
for c in data_df.columns[1:]:   #除去column label 
    if len(data_df[c].unique()) <= discreate_n:
        continue
    else:
        #等密度离散，区间内样本数量相同，若用等长度区间，则pd.cut()
        discreate_df[c] = pd.qcut(data_df[c],discreate_n,labels=False,precision=0,duplicates='drop')


In [35]:
discreate_df.head()     #显示新的离散特征df

blueWins  blueWardsPlaced  blueWardsDestroyed  blueKills  blueDeaths   
0         0                7                   1          6           3  \
1         0                0                   0          2           2   
2         0                2                   0          4           7   
3         0                8                   0          1           2   
4         0                8                   3          3           3   

   blueAssists  blueEliteMonsters  blueDragons  blueHeralds   
0            8                  0            0            0  \
1            3                  0            0            0   
2            2                  1            1            0   
3            3                  1            0            1   
4            4                  0            0            0   

   blueTowersDestroyed  ...  brAssists  brEliteMonsters  brDragons  brHeralds   
0                    0  ...          6                0          0          0  \
1                    0  ...          6               -2         -1         -1   
2                    0  ...          0                1          1          0   
3                    0  ...          1                1          0          1   
4                    0  ...          3               -1         -1          0   

   brTowersDestroyed  brTotalGold  brAvgLevel  brTotalMinionsKilled   
0                  0            6           2                     4  \
1                 -1            1           2                     0   
2                  0            3           2                     2   
3                  0            2           3                     1   
4                  0            3           3                     3   

   brTotalJungleMinionsKilled  brExperienceDiff  
0                           0                 5  
1                           2                 2  
2                           9                 2  
3                           7                 5  
4                           2                 5  

[5 rows x 43 columns]

In [39]:
## 划分数据集
feature_names = discreate_df.columns[1:]
X = discreate_df[feature_names].values
Y = discreate_df['blueWins'].values

train_x,test_x,train_y,test_y = train_test_split(X,Y,test_size=0.2,random_state=RANDOM_SEED)
train_x.shape,test_x.shape,train_y.shape,test_y.shape

((7903, 42), (1976, 42), (7903,), (1976,))

4. 模型定义与训练

In [41]:
Dtree = DecisionTreeClassifier(max_depth=5,min_samples_split=10,criterion='gini')
Dtree.fit(train_x,train_y)
predict_y = Dtree.predict(test_x)
accuracy_scores = accuracy_score(predict_y,test_y)
print(accuracy_scores)

0.7201417004048583


5. 模型调优

In [48]:
%%time
from sklearn.model_selection import GridSearchCV

paramters = {'criterion':['gini','entropy'],
             'max_depth':range(1,6),
             'min_samples_split':[50,100,200,500,1000]}     #定义模型需要遍历的超参数
DT = DecisionTreeClassifier()
grid_search = GridSearchCV(DT,paramters,scoring='accuracy',cv=5,verbose=100,n_jobs=4)   #传入参数
grid_search.fit(train_x,train_y)
print(grid_search.best_score_,grid_search.best_params_)
            

Fitting 5 folds for each of 50 candidates, totalling 250 fits
0.7283313717483726 {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 500}
CPU times: total: 0 ns
Wall time: 2.01 s


In [46]:
#取最佳参数重新训练模型
Dtree = DecisionTreeClassifier(max_depth=5,criterion='gini',min_samples_split=500)
Dtree.fit(train_x,train_y)
predict_y = Dtree.predict(test_x)
accuracy_scores = accuracy_score(predict_y,test_y)
print(accuracy_scores)

0.7231781376518218
